In [ ]:
!/usr/local/cuda/bin/nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2022 NVIDIA Corporation
Built on Wed_Sep_21_10:33:58_PDT_2022
Cuda compilation tools, release 11.8, V11.8.89
Build cuda_11.8.r11.8/compiler.31833905_0


In [ ]:
!pip install git+https://github.com/andreinechaev/nvcc4jupyter.git

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-g24_h7sk
  Running command git clone --filter=blob:none --quiet https://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-g24_h7sk
  Resolved https://github.com/andreinechaev/nvcc4jupyter.git to commit aac710a35f52bb78ab34d2e52517237941399eff
  Preparing metadata (setup.py) ... done
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-py3-none-any.whl size=4287 sha256=786ae1d88bca675534510e052213c966d30ca2eb2e8b6c1b5b7be681bb0d1cc6
  Stored in directory: /tmp/pip-ephem-wheel-cache-o_6osx1t/wheels/a8/b9/18/23f8ef71ceb0f63297dd1903aedd067e6243a68ea756d6feea
Successfully built NVCCPlugin


In [ ]:
%load_ext nvcc_plugin

created output directory at /content/src
Out bin /content/result.out


In [ ]:
%%cuda --name vector_add.cu
#include <cuda_runtime.h>
#include <iostream>

using namespace std;

void fill_array(int* arr, int size) {
    for (int i = 0; i < size; i++) {
        arr[i] = rand() % 100;
    }
}

void add_cpu(int* arr1, int* arr2, int* result, int size) {
    for (int i = 0; i < size; i++) {
        result[i] = arr1[i] + arr2[i];
    }
}

void print_matrix(int* arr, int size) {
    for (int i = 0; i < size; i++) {
        cout << arr[i] << " ";
    }
    cout << endl;
}

__global__ void add(int* arr1, int* arr2, int* arr3, int size) {
    int block_id = blockIdx.x;
    if (block_id < size) {
        arr3[block_id] = arr1[block_id] + arr2[block_id];
    }
}

int main() {
    int* arr1_cpu, * arr2_cpu, * result_cpu;
    int size;
    cout << "Enter size of vector: ";
    cin >> size;

    arr1_cpu = new int[size];
    arr2_cpu = new int[size];
    result_cpu = new int[size];

    fill_array(arr1_cpu, size);
    cout << "Array 1: ";
    print_matrix(arr1_cpu, size);
    fill_array(arr2_cpu, size);
    cout << "Array 2: ";
    print_matrix(arr2_cpu, size);

    int* arr1_gpu, * arr2_gpu, * result_gpu;

    cudaMallocManaged(&arr1_gpu, size * sizeof(int));
    cudaMallocManaged(&arr2_gpu, size * sizeof(int));
    cudaMallocManaged(&result_gpu, size * sizeof(int));

    cudaMemcpy(arr1_gpu, arr1_cpu, size * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(arr2_gpu, arr2_cpu, size * sizeof(int), cudaMemcpyHostToDevice);

    dim3 blockSize(256);
    dim3 gridSize((size + blockSize.x - 1) / blockSize.x);

    add<<<gridSize, blockSize>>>(arr1_gpu, arr2_gpu, result_gpu, size);

    cudaMemcpy(result_cpu, result_gpu, size * sizeof(int), cudaMemcpyDeviceToHost);
    cout << "GPU result:\n";
    print_matrix(result_cpu, size);

    delete[] arr1_cpu;
    delete[] arr2_cpu;
    delete[] result_cpu;
    cudaFree(arr1_gpu);
    cudaFree(arr2_gpu);
    cudaFree(result_gpu);

    return 0;
}


'File written in /content/src/vector_add.cu'

In [ ]:
!nvcc /content/src/vector_add.cu -o /content/src/vector_add


In [ ]:
!/content/src/vector_add

Enter size of vector: 250
Array 1: 83 86 77 15 93 35 86 92 49 21 62 27 90 59 63 26 40 26 72 36 11 68 67 29 82 30 62 23 67 35 29 2 22 58 69 67 93 56 11 42 29 73 21 19 84 37 98 24 15 70 13 26 91 80 56 73 62 70 96 81 5 25 84 27 36 5 46 29 13 57 24 95 82 45 14 67 34 64 43 50 87 8 76 78 88 84 3 51 54 99 32 60 76 68 39 12 26 86 94 39 95 70 34 78 67 1 97 2 17 92 52 56 1 80 86 41 65 89 44 19 40 29 31 17 97 71 81 75 9 27 67 56 97 53 86 65 6 83 19 24 28 71 32 29 3 19 70 68 8 15 40 49 96 23 18 45 46 51 21 55 79 88 64 28 41 50 93 0 34 64 24 14 87 56 43 91 27 65 59 36 32 51 37 28 75 7 74 21 58 95 29 37 35 93 18 28 43 11 28 29 76 4 43 63 13 38 6 40 4 18 28 88 69 17 17 96 24 43 70 83 90 99 72 25 44 90 5 39 54 86 69 82 42 64 97 7 55 4 48 11 22 28 99 43 46 68 40 22 11 10 
Array 2: 5 1 61 30 78 5 20 36 44 26 22 65 8 16 82 58 24 37 62 24 0 36 52 99 79 50 68 71 73 31 81 30 33 94 60 63 99 81 99 96 59 73 13 68 90 95 26 66 84 40 90 84 76 42 36 7 45 56 79 18 87 12 48 72 59 9 36 10 42 87 6 1 13 72 21 55 19 99 

In [ ]:
%%cuda --name matrix_multiply.cu
#include <iostream>
#include <cuda_runtime.h>

using namespace std;

__global__ void matrixMultiply(int* A, int* B, int* C, int m, int n, int k)
{
    int row = blockIdx.y * blockDim.y + threadIdx.y;
    int col = blockIdx.x * blockDim.x + threadIdx.x;

    if (row < m && col < k)
    {
        int sum = 0;
        for (int i = 0; i < n; i++)
        {
            sum += A[row * n + i] * B[i * k + col];
        }
        C[row * k + col] = sum;
    }
}

void printMatrix(int* matrix, int rows, int cols)
{
    for (int i = 0; i < rows; i++)
    {
        for (int j = 0; j < cols; j++)
        {
            cout << matrix[i * cols + j] << " ";
        }
        cout << endl;
    }
}

int main()
{
    int A_rows = 3, A_cols = 2;
    int B_rows = 2, B_cols = 3;

    int A_size = A_rows * A_cols;
    int B_size = B_rows * B_cols;
    int C_size = A_rows * B_cols;

    int* A = new int[A_size];
    int* B = new int[B_size];
    int* C = new int[C_size];

    // Initialize matrix A
    for (int i = 0; i < A_size; i++)
    {
        A[i] = i + 1;
    }

    // Initialize matrix B
    for (int i = 0; i < B_size; i++)
    {
        B[i] = i + 1;
    }

    // Print input matrices
    cout << "Matrix A:" << endl;
    printMatrix(A, A_rows, A_cols);
    cout << "Matrix B:" << endl;
    printMatrix(B, B_rows, B_cols);

    // Allocate memory on the device
    int* dev_A, * dev_B, * dev_C;
    cudaMalloc(&dev_A, A_size * sizeof(int));
    cudaMalloc(&dev_B, B_size * sizeof(int));
    cudaMalloc(&dev_C, C_size * sizeof(int));

    // Copy data from host to device
    cudaMemcpy(dev_A, A, A_size * sizeof(int), cudaMemcpyHostToDevice);
    cudaMemcpy(dev_B, B, B_size * sizeof(int), cudaMemcpyHostToDevice);

    // Define grid and block dimensions
    dim3 dimGrid((B_cols + 16 - 1) / 16, (A_rows + 16 - 1) / 16);
    dim3 dimBlock(16, 16);

    // Launch the matrix multiplication kernel
    matrixMultiply<<<dimGrid, dimBlock>>>(dev_A, dev_B, dev_C, A_rows, A_cols, B_cols);

    // Copy the result from device to host
    cudaMemcpy(C, dev_C, C_size * sizeof(int), cudaMemcpyDeviceToHost);

    // Print the result matrix
    cout << "Matrix C (Result of A * B):" << endl;
    printMatrix(C, A_rows, B_cols);

    // Free memory
    cudaFree(dev_A);
    cudaFree(dev_B);
    cudaFree(dev_C);
    delete[] A;
    delete[] B;
    delete[] C;

    return 0;
}


'File written in /content/src/matrix_multiply.cu'

In [ ]:
!nvcc /content/src/matrix_multiply.cu -o /content/src/matrix_multiply

In [ ]:
!/content/src/matrix_multiply

Matrix A:
1 2 
3 4 
5 6 
Matrix B:
1 2 3 
4 5 6 
Matrix C (Result of A * B):
0 0 0 
0 0 0 
0 0 0 
